# Wavefront sensing with a Shack-Hartmann wavefront sensor

We will simulate a closed-loop adaptive optics system, based on the the Spectro-Polarimetric High-contrast Exoplanet REsearch (SPHERE) adaptive optics (AO) system, that uses a Shack-Hartmann WFS. We will simulate calibration and on-sky operation of this simulated AO system.

We first start by importing the relevant python modules.

In [ ]:
from hcipy import *
from progressbar import progressbar

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import time
import os

%matplotlib inline

## Simulating the VLT pupil

We will model here the VLT pupil, of 8m in diameter (the primary mirror is 8.2m but the actual pupil is restricted by M2 to 8m, there is an extra margin used for choping in the infrared).

In [ ]:
telescope_diameter = 8. #in m
central_obscuration = 1.2 #in m
central_obscuration_ratio = central_obscuration/telescope_diameter
spider_width = 0.05 #5cm
oversizing_factor = 16 / 15

We represent the pupil by a grid of 240px. This is the sampling used by the SH WFS of SPHERE, which is an EMCCD of 240x240px (6px per subapertures, with 40 subapertures on one diameter). 
To avoid some numerical problems at the edges, we oversize the pupil grid by a factor 1.2, e.g. the grid is represented by a grid of 240 * 1.2 = 288 px.

In [ ]:
num_pupil_pixels = 240 * oversizing_factor
pupil_grid_diameter = telescope_diameter * oversizing_factor
pupil_grid = make_pupil_grid(num_pupil_pixels, pupil_grid_diameter)

In [ ]:
VLT_aperture_generator = make_obstructed_circular_aperture(telescope_diameter, \
                        central_obscuration_ratio, num_spiders=4, spider_width=spider_width)

In [ ]:
VLT_aperture = evaluate_supersampled(VLT_aperture_generator,pupil_grid, 4)

The factor 4 indicates that each pixel will be evaluated with 4x supersampling, effectively averaging 4x4=16 subpixels for each pixel.

In [ ]:
imshow_field(VLT_aperture)
plt.xlabel('x position(m)')
plt.ylabel('y position(m)')
plt.colorbar()

As shown above, the pupil is not exactly that of the VLT (the 4 spiders of the VLT intersect on the perimetre of M2, and not at the center), but this is not important here

## Incoming wavefront

Let's assume we work with monochromatic light at 700nm for wavefront sensing and in the K band at 2.2 micron for the scientific channel

In [ ]:
wavelength_wfs = 0.7e-6
wavelength_sci = 2.2e-6
wf = Wavefront(VLT_aperture, wavelength_sci)
wf.total_power = 1

Let visualize the corresponding diffraction pattern. To do so, we need to propagate the wavefront from the pupil to a focal plane. We assume here a perfect lense (see https://docs.hcipy.org/dev/api/hcipy.propagation.FraunhoferPropagator.html for details on the model).
We also need to sample the electric field on a focal plane. We use here 4 pixels per resolution elements and set the field of view to 30 lambda/D in radius at the science wavelength.

In [ ]:
spatial_resolution = wavelength_sci / telescope_diameter
focal_grid = make_focal_grid(q=4, num_airy=30, spatial_resolution=spatial_resolution)

propagator = FraunhoferPropagator(pupil_grid, focal_grid)

unaberrated_PSF = propagator.forward(wf).power
unaberrated_PSF = unaberrated_PSF/unaberrated_PSF.max()

imshow_field(np.log10(unaberrated_PSF))
plt.colorbar()

## Wavefront sensor

The WFS is a squared 40x40 Shack Hartmann WFS. The diameter of the beam needs to be reshaped with a magnifier, otherwise the spots are not resolved by the pupil grid: the spots have a size of f * lambda = 35 micron with a f ratio of 50. If the beam is 5mm, then 1px is 20micron and we resolve the spots.

In [ ]:
f_number = 50
num_lenslets = 40 # 40 lenslets along one diameter
sh_diameter = 5e-3 # m

magnification = sh_diameter / telescope_diameter
magnifier = Magnifier(magnification)

In [ ]:
shwfs = SquareShackHartmannWavefrontSensorOptics(pupil_grid.scaled(magnification), f_number, \
                                                 num_lenslets, sh_diameter)
shwfse = ShackHartmannWavefrontSensorEstimator(shwfs.mla_grid, shwfs.micro_lens_array.mla_index)

We assume a noiseless detector. In practice the EMCCD of SPHERE has RON of about 1 electron.

In [ ]:
camera = NoiselessDetector(focal_grid)

Let's look at the SH image for an undisturbed wavefront

In [ ]:
wf = Wavefront(VLT_aperture, wavelength_wfs)
camera.integrate(shwfs(magnifier(wf)), 1)

image_ref = camera.read_out()

imshow_field(image_ref)
plt.colorbar()

We select subapertures to use for wavefront sensing, based on their flux. The sub-pupils seeing the spiders receive about 75% of the flux of the unobscured sup-apertures. We want to include those, but we do not want to incldues superatures at the edge of the pupil that receive less than 50% of that flux. We therefore use a threshold at 50%.



In [ ]:
fluxes = ndimage.measurements.sum(image_ref, shwfse.mla_index, shwfse.estimation_subapertures)
flux_limit = fluxes.max() * 0.5

estimation_subapertures = np.zeros(shwfs.mla_grid.size)
estimation_subapertures[shwfse.estimation_subapertures[fluxes > flux_limit]] = 1
estimation_subapertures = estimation_subapertures.astype('bool')

shwfse = ShackHartmannWavefrontSensorEstimator(shwfs.mla_grid, shwfs.micro_lens_array.mla_index, estimation_subapertures)

In [ ]:
estimation_subapertures.shape

Calculate reference slopes.

In [ ]:
slopes_ref = shwfse.estimate([image_ref])

## Deformable mirror

Let's assume we control 500 disk harmonic modes with the DM.

In [ ]:
num_modes = 500
dm_modes = make_disk_harmonic_basis(pupil_grid, num_modes, telescope_diameter, 'neumann')
dm_modes = ModeBasis([mode / np.ptp(mode) for mode in dm_modes], pupil_grid)
deformable_mirror = DeformableMirror(dm_modes)

Then we need to calibrate the interaction matrix: you excite individually each mode of the DM and estimate the centroids of the spots

In [ ]:
for i in progressbar(range(10)):
#for i in progressbar(range(num_modes)):
    ## Extract the centers of the lenslets
    act_levels = np.zeros(num_modes)
    act_levels[i] = 1e-7
    deformable_mirror.actuators = act_levels
    dm_wf = deformable_mirror(wf)
    sh_wf = shwfs(magnifier(dm_wf))
    sh_img = sh_wf.power
    relative_offsets = np.array(shwfse.estimate([sh_img]))
    spot_centers = np.copy(relative_offsets)
    # Returned spot centers by shwfse are relative to the center of the microlens.
    # Add the center of the microlens back on to get absolute positions.
    spot_centers[0,:] += shwfs.mla_grid.subset(shwfse.estimation_subapertures).x
    spot_centers[1,:] += shwfs.mla_grid.subset(shwfse.estimation_subapertures).y

    # Plot the DM shape and WFS image
    fig = plt.figure(figsize=(30,15))
    plt.subplot(1,2,1)
    plt.title('Mode {0:d}: DM shape'.format(i+1))
    im1 = imshow_field(deformable_mirror.surface, vmin=-1e-7, vmax=1e-7, cmap='bwr')

    plt.subplot(1,2,2)
    plt.title('Mode {0:d}: SH image'.format(i+1))
    im2 = imshow_field(sh_img)
    #plt.plot(spot_centers[0,:], spot_centers[1,:], 'r,')
    plt.quiver(shwfs.mla_grid.subset(shwfse.estimation_subapertures).x,
        shwfs.mla_grid.subset(shwfse.estimation_subapertures).y,
        relative_offsets[0,:], relative_offsets[1,:],
        color='white')
    plt.draw()

## Calibrating the interaction matrix

Now we'll build the interaction matrix by exciting each mode individually and record the spot displacements 

In [ ]:
probe_amp = 0.01 * wavelength_wfs
response_matrix = []

wf = Wavefront(VLT_aperture, wavelength_wfs)
wf.total_power = 1

for ind in progressbar(range(num_modes)):
    slope = 0

    # Probe the phase response
    for s in [1, -1]:
        amp = np.zeros((num_modes,))
        amp[ind] = s * probe_amp
        deformable_mirror.actuators = amp

        dm_wf = deformable_mirror.forward(wf)
        wfs_wf = shwfs(magnifier(dm_wf))

        camera.integrate(wfs_wf, 1)
        image = camera.read_out()
        
        slopes = shwfse.estimate([image])

        slope += s * slopes / (2 * probe_amp)

    response_matrix.append(slope.ravel())

response_matrix = ModeBasis(response_matrix)

We invert the interaction matrix using Tikhonov regularisation

In [ ]:
rcond = 1e-3
reconstruction_matrix = inverse_tikhonov(response_matrix.transformation_matrix, rcond=rcond)
print(reconstruction_matrix.shape)

We initialise the DM with a random position by setting the DM actuators with a random white noise of RMS 5% of the WFS wavelength

In [ ]:
deformable_mirror.random(0.05 * wavelength_wfs)
imshow_field(deformable_mirror.phase_for(wavelength_wfs))

## Closing the loop without atmospheric disturbance

In [ ]:
zero_magnitude_flux = 3.9e10 #3.9e10 photon/s for a mag 0 star
stellar_magnitude = 0
delta_t = 1e-3 # 1kHZ means 1ms per loop iteration

wf_wfs = Wavefront(VLT_aperture, wavelength_wfs)
wf_sci = Wavefront(VLT_aperture, wavelength_sci)

wf_wfs.total_power = zero_magnitude_flux * 10**(-stellar_magnitude / 2.5)
print("Photon flux per frame {:g}".format(wf_wfs.total_power * delta_t))
wf_sci.total_power = zero_magnitude_flux * 10**(-stellar_magnitude / 2.5)

undisturbed_PSF_wfs = propagator.forward(wf_wfs).power
max_undisturbed_PSF_wfs = undisturbed_PSF_wfs.max()
undisturbed_PSF_sci = propagator.forward(wf_sci).power
max_undisturbed_PSF_sci = undisturbed_PSF_sci.max()

In [ ]:
deformable_mirror.random(0.05 * wavelength_wfs)

gain = 0.3
leakage = 0.01
num_iterations = 10 

long_exposure = 0

# Set up animation
plt.figure(figsize=(8,8))
anim = FFMpegWriter('AO_simulation_without_turbulence.mp4', framerate=10)

for timestep in progressbar(range(num_iterations)):

    # Propagate through deformable mirror.
    wf_after_dm = deformable_mirror(wf_wfs)

    # Propagate through SH-WFS
    wf_after_sh = shwfs(magnifier(wf_after_dm))

    # Propagate the NIR wavefront 
    wf_sci_after_dm = propagator(deformable_mirror(wf_sci))

    # Read out WFS camera
    camera.integrate(wf_after_sh, delta_t)
    wfs_image = camera.read_out()
    wfs_image = large_poisson(wfs_image).astype('float')
    
    # Accumulate long-exposure image
    long_exposure += wf_sci_after_dm.power / num_iterations

    # Calculate slopes from WFS image
    slopes = shwfse.estimate([wfs_image + 1e-10])
    slopes -= slopes_ref
    slopes = slopes.ravel()

    # Perform wavefront control and set DM actuators
    deformable_mirror.actuators = (1 - leakage) * deformable_mirror.actuators - gain * reconstruction_matrix.dot(slopes)

    # Plotting
    plt.clf()
    plt.suptitle('Timestep %d / %d' % (timestep, num_iterations))

    plt.subplot(2,2,1)
    plt.title('DM surface [$\\mu$m]')
    imshow_field(deformable_mirror.surface * 1e6, cmap='RdBu', vmin=-0.2, vmax=0.2)
    plt.colorbar()

    plt.subplot(2,2,2)
    plt.title('WFS image [counts]')
    imshow_field(wfs_image, cmap='inferno')
    plt.colorbar()

    plt.subplot(2,2,3)
    plt.title('Instantaneous PSF at 2.2$\\mu$m [log]')
    imshow_field(np.log10(wf_sci_after_dm.power/ wf_sci.power.max()), vmin=-3, vmax=0, cmap='inferno') #
    plt.colorbar()

    plt.subplot(2,2,4)
    plt.title('Average PSF at 2.2$\\mu$m [log]')
    imshow_field(np.log10(long_exposure / long_exposure.max()), vmin=-5, cmap='inferno')
    plt.colorbar()

    anim.add_frame()

plt.close()
anim.close()

# Show created animation
anim

Exercise: how would you compute the Strehl ? 

In [ ]:
strehl = long_exposure[np.argmax(undisturbed_PSF_sci)] / np.max(undisturbed_PSF_sci)
print('Strehl = {0:.1f}%'.format(strehl * 100))

## Simulating the atmosphere

In [ ]:
seeing = 0.6 # [arcsec] @ 500nm (convention)
fried_parameter = 500e-9 / np.deg2rad(seeing / 3600) 
print('r0   = {0:.1f}cm'.format(fried_parameter * 100))

Cn_squared = Cn_squared_from_fried_parameter(fried_parameter, 500e-9)

outer_scale = 40 # [meter]
print('L0   = {0:.1f}m'.format(outer_scale))

tau0 = 0.005 # [sec]
print('tau0 = {0:.1f}ms'.format(tau0 * 1000))

velocity = 0.314 * fried_parameter / tau0
print('v    = {0:.1f}m/s'.format(velocity))

In [ ]:
layer = InfiniteAtmosphericLayer(pupil_grid, Cn_squared, outer_scale, velocity)

phase_screen_grad = layer.phase_for(wavelength_wfs) # in radian
phase_screen_microns = phase_screen_grad * (wavelength_wfs / 1e-6) / (2 * np.pi)

In [ ]:
plt.figure(figsize=(5, 4))
anim = FFMpegWriter('atmospheric_turbulence.mp4', framerate=10)

num_time_steps = 10
for timestep in progressbar(range(num_time_steps)):
    layer.t = timestep * delta_t
    
    plt.clf()
    plt.suptitle('Timestep %d / %d' % (timestep, num_iterations))

    plt.subplot(1,1,1)
    plt.title('Turbulent wavefront [$\\mu$m]')
    imshow_field(layer.phase_for(wavelength_wfs) * (wavelength_wfs / 1e-6) / (2 * np.pi), 
        vmin=-6, vmax=6, cmap='RdBu')
    plt.colorbar()
    
    anim.add_frame()

plt.close()
anim.close()

# Show created animation
anim

## Closing the loop on-sky

In [ ]:
deformable_mirror.flatten()
gain = 0.3
leakage = 0.01
num_iterations = 1000
burn_in_iterations = 5

coro = PerfectCoronagraph(VLT_aperture, 4)

long_exposure = focal_grid.zeros()
long_exposure_coro = focal_grid.zeros()

# Set up animation
plt.figure(figsize=(8, 8))
anim = FFMpegWriter('AO_simulation_with_turbulence.mp4', framerate=10)

for timestep in progressbar(range(num_iterations)):
    layer.t=timestep*delta_t

    # Propagate through atmosphere and deformable mirror.
    wf_wfs_after_atmos = layer(wf_wfs)
    wf_wfs_after_dm = deformable_mirror(wf_wfs_after_atmos)

    # Propagate through SH-WFS
    wf_wfs_on_sh = shwfs(magnifier(wf_wfs_after_dm))

    # Propagate the NIR wavefront 
    wf_sci_focal_plane = propagator(deformable_mirror(layer(wf_sci)))
    wf_sci_coro = propagator(coro(deformable_mirror(layer(wf_sci))))

    # Read out WFS camera
    camera.integrate(wf_wfs_on_sh, delta_t)
    wfs_image = camera.read_out()
    wfs_image = large_poisson(wfs_image).astype('float')
    
    # Accumulate long-exposure image
    if timestep >= burn_in_iterations:
        long_exposure += wf_sci_focal_plane.power / (num_iterations - burn_in_iterations)
        long_exposure_coro += wf_sci_coro.power / (num_iterations - burn_in_iterations)

    # Calculate slopes from WFS image
    slopes = shwfse.estimate([wfs_image + 1e-10])
    slopes -= slopes_ref
    slopes = slopes.ravel()

    # Perform wavefront control and set DM actuators
    deformable_mirror.actuators = (1 - leakage) * deformable_mirror.actuators - gain * reconstruction_matrix.dot(slopes)

    # Plotting
    if timestep % 10 == 0:
        plt.clf()

        plt.suptitle('Timestep %d / %d' % (timestep, num_iterations))

        plt.subplot(2,2,1)
        plt.title('DM surface [$\\mu$m]')
        imshow_field(deformable_mirror.surface * 1e6, cmap='RdBu', vmin=-2, vmax=2, mask=VLT_aperture)
        plt.colorbar()

        plt.subplot(2,2,2)
        plt.title('WFS image [counts]')
        imshow_field(wfs_image, cmap='inferno')
        plt.colorbar()

        plt.subplot(2,2,3)
        plt.title('Instantaneous PSF at 2.2$\\mu$m [log]')
        imshow_field(np.log10(wf_sci_focal_plane.power/ wf_sci_focal_plane.power.max()), vmin=-3, vmax=0, cmap='inferno') #
        plt.colorbar()

        if timestep >= burn_in_iterations:
            plt.subplot(2,2,4)
            plt.title('Average PSF at 2.2$\\mu$m [log]')
            imshow_field(np.log10(long_exposure_coro / long_exposure.max()), vmin=-5, cmap='inferno')
            plt.colorbar()

        anim.add_frame()

plt.close()
anim.close()

# Show created animation
anim

In [ ]:
strehl = long_exposure[np.argmax(undisturbed_PSF_sci)] / np.max(undisturbed_PSF_sci)
print('Strehl = {0:.1f}%'.format(strehl * 100))

In [ ]:
# Cleanup created movie files
os.remove('AO_simulation_without_turbulence.mp4')
os.remove('atmospheric_turbulence.mp4')
os.remove('AO_simulation_with_turbulence.mp4')